In [14]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [15]:
df = pd.read_csv('IEA-EV-dataEV_salesCarsHistorical.csv')

data = df.drop(['category', 'parameter', 'mode', 'unit'], axis=1)

encoded_data = pd.get_dummies(data, columns=['region', 'powertrain'])

X = encoded_data.drop('value', axis=1)
y = encoded_data['value']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor()
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [16]:
latest_year = data['year'].max()

In [17]:
future_data = pd.DataFrame({
'year': [latest_year + 1, latest_year + 2, latest_year + 3],
'region': [data['region'].unique()[0]] * 3,
'powertrain': [data['powertrain'].unique()[0]] * 3
})

In [18]:
encoded_future_data = pd.get_dummies(future_data, columns=['region', 'powertrain'])

In [19]:
encoded_future_data = encoded_future_data.reindex(columns=encoded_data.columns, fill_value=0)

In [20]:
encoded_future_data = encoded_future_data.drop('value', axis=1)

In [21]:
future_predictions = model.predict(encoded_future_data)

In [22]:
future_dataset = pd.DataFrame({
'year': future_data['year'],
'region': future_data['region'],
'powertrain': future_data['powertrain'],
'value': future_predictions
})

In [23]:
print(future_dataset)

   year     region powertrain         value
0  2023  Australia        BEV  33813.496094
1  2024  Australia        BEV  33813.496094
2  2025  Australia        BEV  33813.496094


In [24]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

In [25]:
mse

9944876330.993235